In [4]:
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.stem.lancaster import LancasterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.probability import FreqDist
import statsmodels.api as sm
from sklearn import model_selection

#nltk.download() #<---Use this command to download WordNet in corpora

In [2]:
train = pd.read_csv("/media/fexunexa/Tree/Kaggle/Git Quora/data/train.csv")

#### Vendo as 3 primeiras informações

In [22]:
train.head(3)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0


In [23]:
train.shape

(404290, 6)

#### Selecionando linhas

In [26]:
train[4:6]

,id,qid1,qid2,question1,question2,is_duplicate
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0
5,5,11,12,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan...",1


#### Selecionando Coluna

In [32]:
train['question1'][200:205]

200    What is the easiest way to become a billionair...
201    Are there People who are willing to give free ...
202      What's the best way to start learning robotics?
203                  Why do people hate Hillary Clinton?
204    Why are women who are on their periods are reg...
Name: question1, dtype: object

#### Selecionar duas colunas 

In [46]:
train.loc[1:5,['question1','question2']]

,question1,question2
1,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?
5,Astrology: I am a Capricorn Sun Cap moon and c...,"I'm a triple Capricorn (Sun, Moon and ascendan..."


#### Selecionando pela posição

In [47]:
train.iloc[1:5,2:4]

,qid2,question1
1,4,What is the story of Kohinoor (Koh-i-Noor) Dia...
2,6,How can I increase the speed of my internet co...
3,8,Why am I mentally very lonely? How can I solve...
4,10,"Which one dissolve in water quikly sugar, salt..."


#### Selecionando informação específica

In [49]:
train[train.is_duplicate == 1].shape

(149263, 6)

#### Verificando por vazios

In [119]:
vazioQ1 = []
vazioQ2 = []

for i in range(0,len(train.question2)):
    if type(train.question2[i]) is float:
        vazioQ2.append(i)
    if type(train.question1[i]) is float:
        vazioQ1.append(i)

In [120]:
print(vazioQ1, vazioQ2)

[] [105780, 201841]


#### Lower Case and removing punctuation

In [19]:
def lower_rm_punct(phrase):
    if type(phrase) is not float:
        phrase = phrase.lower()
        phrase = phrase.translate(str.maketrans('','', string.punctuation))
    
    return phrase

In [20]:
train.question1 = train.question1.apply(lambda x: lower_rm_punct(x))
train.question2 = train.question2.apply(lambda x: lower_rm_punct(x))

#### Function to Remove stopwords

In [46]:
def rm_stopwords(phrase):
    stop = set(stopwords.words('english'))
    
    if type(phrase) is not float:
        phrase = [i for i in phrase.split() if i not in stop]
        phrase = ' '.join(phrase)
    return phrase

In [47]:
train_stopw = train

train_stopw.question1 = train.question1.apply(lambda x: rm_stopwords(x))
train_stopw.question2 = train.question2.apply(lambda x: rm_stopwords(x))

#### Function to "stem"

#### From Stackoverflow:

##### At the very basics of it, the major difference between the porter and lancaster stemming algorithms is that the lancaster stemmer is significantly more aggressive than the porter stemmer. The three major stemming algorithms in use today are Porter, Snowball(Porter2), and Lancaster (Paice-Husk), with the aggressiveness continuum basically following along those same lines. Porter is the least aggressive algorithm, with the specifics of each algorithm actually being fairly lengthy and technical. Here is a break down for you though:

##### Porter: Most commonly used stemmer without a doubt, also one of the most gentle stemmers. One of the few stemmers that actually has Java support which is a plus, though it is also the most computationally intensive of the algorithms(Granted not by a very significant margin). It is also the oldest stemming algorithm by a large margin.

##### Porter2: Nearly universally regarded as an improvement over porter, and for good reason. Porter himself in fact admits that it is better than his original algorithm. Slightly faster computation time than porter, with a fairly large community around it.

##### Lancaster: Very aggressive stemming algorithm, sometimes to a fault. With porter and snowball, the stemmed representations are usually fairly intuitive to a reader, not so with Lancaster, as many shorter words will become totally obfuscated. The fastest algorithm here, and will reduce your working set of words hugely, but if you want more distinction, not the tool you would want.

##### Honestly, I feel that Snowball is usually the way to go. There are certain circumstances in which Lancaster will hugely trim down your working set, which can be very useful, however the marginal speed increase over snowball in my opinion is not worth the lack of precision. Porter has the most implementations though and so is usually the default go-to algorithm, but if you can, use snowball.


In [22]:
def stem_lancaster(phrase):
    LS = LancasterStemmer()
    
    if type(phrase) is not float:
        phrase = [LS.stem(i) for i in phrase.split()]
        phrase = ' '.join(phrase)
    return phrase


#### Function to "lemmatize"

In [23]:
def lemm_wordnet(phrase):
    lemm = WordNetLemmatizer()

    if type(phrase) is not float:
        phrase = [lemm.lemmatize(i) for i in phrase.split()]
        phrase = ' '.join(phrase)
    return phrase

#### Some Numbers

In [43]:
def freq_all_text(question1,question2): #Pass all question

    question1_freq = []
    question2_freq = []
    for i in range(0,len(question1)):
        question1_freq += question1[i].split()
        question2_freq += str(question2[i]).split()
    
    len_question_freq = len(set(question1_freq + question2_freq))
    
    question1_freq = FreqDist(question1_freq)
    question2_freq = FreqDist(question2_freq)

    return question1_freq, question2_freq, len_question_freq

def percentage_by_text(phrase1, phrase2, question1_freq, question2_freq, len_question_freq):
    phrase2 = str(phrase2)
    
    for i in range(0, len(phrase1)):
            less_common_phrase1 = [i for i in set(phrase1.split()) if i not in phrase2.split()]
            less_common_phrase2 = [i for i in set(phrase2.split()) if i not in phrase1.split()]
            
    freq_less_common_phrase1 = 0
    freq_less_common_phrase2 = 0
            
    for i in less_common_phrase1:
        freq_less_common_phrase1 += question1_freq[i]
    
    freq_less_common_phrase1 = freq_less_common_phrase1/len_question_freq
    
    for i in less_common_phrase2:
        freq_less_common_phrase2 += question2_freq[i]
    
    freq_less_common_phrase2 = freq_less_common_phrase2/len_question_freq
    
    if freq_less_common_phrase1 == 0:
        return freq_less_common_phrase2
    
    elif freq_less_common_phrase2 == 0:
        return freq_less_common_phrase1
    
    elif freq_less_common_phrase1 >= freq_less_common_phrase2:
        return freq_less_common_phrase2
    
    else:
        return freq_less_common_phrase1

def percentages_by_phrase(phrase1,phrase2):
    phrase2 = str(phrase2)
    
    #Percentage between numbers of words
    if len(phrase1.split()) >= len(phrase2.split()):
        dif_wdsize = len(phrase2.split())/len(phrase1.split())
    else:
        dif_wdsize = len(phrase1.split())/len(phrase2.split())
   
    #Percentage between numbers of letters
    if len(phrase1) >= len(phrase2):
        dif_ltsize = len(phrase2)/len(phrase1)
    else:
        dif_ltsize = len(phrase1)/len(phrase2)
        
    #Percentage between letters and words per phrase
    dif_ltwdsize = (len(phrase1.split())/len(phrase1))/(len(phrase2.split())/len(phrase2))

    #Percentage of equal words
    equal = [i for i in set(phrase1.split()) if i in set(phrase2.split())]
    equal = ' '.join(equal)
    
    unique = set((phrase1 + ' ' + phrase2).split()) #Counting all unique words between the 2 phrases
    unique = ' '.join(unique)
    
    perc_equal_w = len(equal)/len(unique)
    
    return dif_wdsize, dif_ltsize, dif_ltwdsize, perc_equal_w

# Teste

In [ ]:
train = train[0:300]

#### Saving some new informations

In [28]:
for i in range(0, len(train.question1)):
    p_size_word, p_size_letters, p_word_letters, p_equal_w = percentages_by_phrase(train.loc[i,'question1'],train.loc[i,'question2'])
    
    train.loc[i, 'p_size_word'] = p_size_word
    train.loc[i, 'p_size_letters'] = p_size_letters
    train.loc[i, 'p_word_letters'] = p_word_letters
    train.loc[i, 'p_equal_w'] = p_equal_w


In [35]:
question1_freq, question2_freq, len_question_freq = freq_all_text(train.question1,train.question2)

In [44]:
for i in range(0, len(train.question1)):
    
    p_less_common = percentage_by_text(train.loc[i,'question1'],train.loc[i,'question2'], question1_freq, question2_freq, len_question_freq)
    
    train.loc[i, 'p_less_common'] = p_less_common

#### Logistic Regression

In [75]:
logit = sm.Logit(train['is_duplicate'], train.loc[:,['p_less_common','p_equal_w','p_word_letters']])

# fit the model
result = logit.fit()
result.summary()

Optimization terminated successfully.
         Current function value: 0.611042
         Iterations 5


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:           is_duplicate   No. Observations:                  300
Model:                          Logit   Df Residuals:                      297
Method:                           MLE   Df Model:                            2
Date:                Mon, 20 Mar 2017   Pseudo R-squ.:                 0.08422
Time:                        02:30:23   Log-Likelihood:                -183.31
converged:                       True   LL-Null:                       -200.17
                                        LLR p-value:                 4.774e-08
==================================================================================
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
p_less_common     -2.1223      3.207     -0.662      0.508      -8.407       4.162
p_equal_w          2.6621      0.582      4.571      0.000       1.521       3.803
p_word_letters    -1.3403      0.335     -3.998      0.000      -1.997      -0.683
==================================================================================
"""

In [76]:
result.conf_int()

,0,1
p_less_common,-8.407121,4.162432
p_equal_w,1.520682,3.803491
p_word_letters,-1.997304,-0.683256


In [77]:
result.predict(train.loc[:,['p_less_common','p_equal_w','p_word_letters']])

0      0.735484
1      0.326307
2      0.265023
3      0.099993
4      0.318818
5      0.375550
6      0.146736
7      0.308781
8      0.610755
9      0.383125
10     0.200228
11     0.456504
12     0.609740
13     0.721750
14     0.739665
15     0.285728
16     0.594740
17     0.171516
18     0.352996
19     0.540069
20     0.316021
21     0.409747
22     0.335203
23     0.083871
24     0.155671
25     0.606983
26     0.510538
27     0.308631
28     0.644329
29     0.373665
         ...   
270    0.733909
271    0.294089
272    0.205795
273    0.288837
274    0.513602
275    0.397372
276    0.192514
277    0.680053
278    0.391363
279    0.366511
280    0.208913
281    0.367054
282    0.289876
283    0.123947
284    0.446840
285    0.429860
286    0.422897
287    0.643417
288    0.364285
289    0.402311
290    0.730083
291    0.639781
292    0.272389
293    0.600580
294    0.277196
295    0.472287
296    0.381659
297    0.125428
298    0.753771
299    0.436974
dtype: float64